<a href="https://colab.research.google.com/github/ZizZu94/resnet-grad-cam/blob/main/resnet_grad_cam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>